In [23]:
import os
from openai import OpenAI

PROJECT_ID = "proj_hUizl3mrZGSfmp4C6DI60dJo"
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
model = 'gpt-4o-mini'

In [8]:
import pypdf

document_filename = "../tests/data/bank_statement_1.pdf"

bank_statement_text = ""
with open(document_filename, "rb") as pdf_file:
    read_pdf = pypdf.PdfReader(pdf_file)
    for page in read_pdf.pages:
        bank_statement_text += page.extract_text()

print(bank_statement_text)
with open("../tests/data/bank_statement_1.txt", 'w') as f:
    f.write(bank_statement_text)

Bank 1 of Testing
Customer Support: 1-800-555-1234
www.fakebankdomain.com
Account Holder: John Doe 1
Account Number: XXXX-XXXX-XXXX-6781
Statement Period: 2023-01
Date Description Debit ($) Credit ($)
01/01/2023 Direct Deposit 326.73
20/01/2023 Direct Deposit 424.61
04/01/2023 ACH Payment 215.70
03/01/2023 Check Deposit 464.84
20/01/2023 Debit Card Purchase 695.38
04/01/2023 POS Purchase 136.10
27/01/2023 Online Transfer 285.34
05/01/2023 ACH Payment 436.96
16/01/2023 POS Purchase 203.32
15/01/2023 POS Purchase 613.07
28/01/2023 POS Purchase 189.97
07/01/2023 Debit Card Purchase 65.51
08/01/2023 Check Deposit 592.13
20/01/2023 Debit Card Purchase 289.50
28/01/2023 POS Purchase 10.44
24/01/2023 Loan Repayment 541.03
Bank 1 - Confidential Statement | Page 1Bank 1 of Testing
Customer Support: 1-800-555-1234
www.fakebankdomain.com
End of Statement
Bank 1 - Confidential Statement | Page 2


In [9]:
document_filename = "../files/invoice_1.pdf"

invoice_text = ""
with open(document_filename, "rb") as pdf_file:
    read_pdf = pypdf.PdfReader(pdf_file)
    for page in read_pdf.pages:
        invoice_text += page.extract_text()

print(invoice_text)
with open("../tests/data/invoice_1.txt", 'w') as f:
    f.write(invoice_text)

 
 
 
 
 
Company Name Customer Company Name 
Address Contact Name 
Phone  Address 
Email Phone 
Website Email 
 
 
Invoice Number: 1234 Issue Date: Date Expiry Date: Date 
 
 
Item # Description Quantity Unit Price Total 
     
     
     
     
     
     
     
     
 
 Subtotal  
 Tax  
 
 TOTAL  
 
 I N V O I C E 


In [11]:
import pytesseract
from PIL import Image

image_path = "../files/drivers_license_1.jpg"
image = Image.open(image_path)
drivers_licence_text = pytesseract.image_to_string(image)

print(drivers_licence_text)
with open("../tests/data/drivers_licence_1.txt", 'w') as f:
    f.write(drivers_licence_text)

(ANY STATE

DRIVER LICENSE

License No. P99999999 Expires 00-00-00

JOE A SAMPLE
123 ANY STREET
ANYTOWN, ANY STATE 99999
4 Sex: M Hair: Black
Ht: 6-03 Wt: 200

Eyes: Brown DOB: 01-01-81 ©




In [42]:
few_shot_prompt = f"""
You are an AI model that classifies extracted text from documents into four categories: "bank statement," "invoice," "driver license," or "unknown." Below are examples of text for each type (except for "unknown"). Use these examples to classify new inputs. Your response should be in JSON format, indicating the type of document.

**Examples:**

1. **Example 1: Bank Statement**
   {bank_statement_text}

   **Output:**
   {{
     "type": "bank statement"
   }}

2. **Example 2: Invoice**
   {invoice_text}

   **Output:**
   {{
     "type": "invoice"
   }}

3. **Example 3: Driver License**
   {drivers_licence_text}

   **Output:**
   {{
     "type": "driver license"
   }}
"""

In [45]:
import json
from pydantic import BaseModel

class StatementClassification(BaseModel):
    type: str
class CustomException(Exception):
    pass
def classify_document_fewshot_openai(text):
    messages = []
    messages.append({
        "role": "system",
        "content": few_shot_prompt
    })
    
    messages.append({
        "role": "user",
        "content": "Classify the following text:\n" + text
        })
    try:
        response = client.beta.chat.completions.parse(model=model, messages=messages, response_format=StatementClassification)
    except Exception as e:
        raise CustomException("OpenAI call failed") from e

    print(response.choices[0].message)

    try:
        output = response.choices[0].message.parsed
        #structured_output = json.loads(output)['type'].lower()
    except Exception as e:
        raise CustomException("OpenAI formatting incorrect", response) from e

    if output in ['bank statement', 'invoice', 'driver licence', 'unknown']:
        return output
    else:
        print(output)
    return "unknown"
    

In [46]:
result = classify_document_fewshot_openai(bank_statement_text)
print(result)

ParsedChatCompletionMessage[StatementClassification](content='{\n  "type": "bank statement"\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=StatementClassification(type='bank statement'))
type='bank statement'
unknown
